<a href="https://colab.research.google.com/github/JohanSaenz/Clase-IA/blob/main/Tarea1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LECTURA DE LOS DATOS


In [15]:
import numpy as np
import os
import gzip

def load_set(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' % tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' % tipo)
    
  with gzip.open(ruta_categorias, 'rb') as rut_cat:
      etiquetas = np.frombuffer(rut_cat.read(), dtype=np.uint8, offset=8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
      imagenes = np.frombuffer(rut_imgs.read(), dtype=np.uint8, offset=16).reshape(len(etiquetas), 784)

  return imagenes, etiquetas
  

## ENLAZANDO DRIVE CON COLAB

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

ruta = '/content/gdrive/MyDrive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = load_set(ruta, tipo='train')
X_test, Y_test = load_set(ruta, tipo='test')

Mounted at /content/gdrive


## AJUSTAR LOS DATOS A MULTIPLOS DE 128

In [16]:
X_train = X_train[0:59904:]
X_test = X_test[0:9984:]
Y_train = Y_train[0:59904:]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train,(59904,28,28,1))
X_test = np.reshape(X_test,(9984,28,28,1))

## IMPORTAR Tensorflow 2 (YA INCLUYE KERAS)

In [17]:
%tensorflow_version 2.x
import tensorflow as tf

print('Versión de Tensorflow: ' + tf.__version__)

Versión de Tensorflow: 2.8.0


## CREACIÓN DEL MÓDELO

In [12]:
tf.random.set_seed(200)

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))

#Imprimir pantalla de información del módelo
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 28, 28, 1)        4         
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 14, 14, 64)       256       
 hNormalization)                                                 
                                                      

## ENTRENAMIENTO CON CPU

In [13]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

### Dos iteraciones con CPU

In [23]:
import timeit 


def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  
  return None
cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')

Epoch 1/2
468/468 [==============================] - 744s 2s/step - loss: 0.0495 - accuracy: 0.9860 - val_loss: 0.0520 - val_accuracy: 0.9884
Epoch 2/2
468/468 [==============================] - 739s 2s/step - loss: 0.0346 - accuracy: 0.9896 - val_loss: 0.0557 - val_accuracy: 0.9856


### Tiempo de entremamiento con CPU

In [24]:
print("Tiempo de entrenamiento " + str(cpu_time) + " segundos")

Tiempo de entrenamiento 1522.143867061 segundos


#VERIFICA LA DISPONIBILIDAD DE LA GPU

In [15]:
nombre_gpu= tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU NO ENCONTRADA')
print('GPU ENCONTRADA: {}'.format(nombre_gpu))  

GPU ENCONTRADA: /device:GPU:0


# ENTRENAMIENTO CON GPU

In [25]:
import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    model.fit(X_train,Y_train,validation_data=(X_test,Y_test), batch_size=128, epochs=2, verbose=1)
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')


Epoch 1/2
468/468 [==============================] - 19s 40ms/step - loss: 0.0845 - accuracy: 0.9796 - val_loss: 0.0574 - val_accuracy: 0.9844
Epoch 2/2
468/468 [==============================] - 18s 39ms/step - loss: 0.0574 - accuracy: 0.9850 - val_loss: 0.0489 - val_accuracy: 0.9886


### Tiempo de entremamiento con GPU

In [26]:
print("Tiempo de entrenamiento " + str(gpu_time) + " segundos")

Tiempo de entrenamiento 36.83116315400002 segundos


# ENTRENAMIENTO CON TPU 

### Configurar la TPU


In [18]:
%tensorflow_version 2.x
import tensorflow as tf
import os

try:
  tpu=tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  print('TPU ENCONTRADA ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: TPU ESTÁ PERDIDA') 

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tf_strategy = tf.distribute.experimental.TPUStrategy(tpu)

TPU ENCONTRADA  ['10.19.120.250:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.19.120.250:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.19.120.250:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# IMPLEMENTACIÓN DEL MÓDELO UTILIZANDO "tpu_strategy"

In [30]:
from tensorflow.python.distribute import tpu_strategy
tf.random.set_seed(200)

with tpu_strategy.maybe_init_scope():
  modelo = tf.keras.models.Sequential()

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
  modelo.add(tf.keras.layers.Conv2D(256, (5, 5), padding='same', activation='elu'))
  modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
  modelo.add(tf.keras.layers.Dropout(0.25))

  modelo.add(tf.keras.layers.Flatten())
  modelo.add(tf.keras.layers.Dense(256))
  modelo.add(tf.keras.layers.Activation('elu'))
  modelo.add(tf.keras.layers.Dropout(0.5))
  modelo.add(tf.keras.layers.Dense(10))
  modelo.add(tf.keras.layers.Activation('softmax'))
  
  #Correr el módelo e imprimir la información en la pantalla
  modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  modelo.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_9 (Batc  (None, 28, 28, 1)        4         
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 64)        1664      
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 14, 14, 64)        0         
                                                                 
 batch_normalization_10 (Bat  (None, 14, 14, 64)       256       
 chNormalization)                                                
                                                      

### Convertir los datos a floats

In [31]:
x_train = X_train.astype(np.float32)
y_train = Y_train.astype(np.float32)
x_test = X_test.astype(np.float32)
y_test = Y_test.astype(np.float32)

### Iteraciones Utilizando TPU


In [32]:
import timeit

def entrenamiento_tpu():
  modelo.fit(x_train,y_train,validation_data=(x_test,y_test), batch_size=128, epochs=2, verbose=1)
  return None

tpu_time = timeit.timeit('entrenamiento_tpu()', number=1, setup='from __main__ import entrenamiento_tpu')

Epoch 1/2
468/468 [==============================] - 41s 87ms/step - loss: 0.3540 - accuracy: 0.9109 - val_loss: 0.0616 - val_accuracy: 0.9826
Epoch 2/2
468/468 [==============================] - 40s 86ms/step - loss: 0.1030 - accuracy: 0.9724 - val_loss: 0.0401 - val_accuracy: 0.9889


### Tiempo de entrenamiento con TPU

In [33]:
print("Tiempo de entrenamiento " + str(tpu_time) + " segundos")

Tiempo de entrenamiento 83.72487979200014 segundos
